**Import Libraries**

In [ ]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import numpy as np

**Load DataFrames**

In [ ]:
##AirBnB Manifest#####################################################################################################

airbnb = pd.read_csv('Cleaning/Airbnbavmonth.csv')

##Madrid Housing#######################################################################################################

housing = pd.read_csv('Cleaning/MadridHouseMarket.csv')


##Madrid Housing outliers limited######################################################################################

madridcut = pd.read_csv('Cleaning/madrid_no_outliers.csv')

###Import Csv of lat_long for use with maps

latlong = pd.read_csv('Cleaning/cleaned_data/latlong.csv')

**Dashboard**

In [ ]:
####################DETERMINE GROUPBYS IN ORDER TO MEASURE STATS#######################################################
madridcutairb = madridcut[madridcut['type'] == 'airbnb']
madridcutsale = madridcut[madridcut['type'] == 'sale']
madridcutrent = madridcut[madridcut['type'] == 'rent']

#Individual AirBnB Listings
airgroup = madridcutairb.groupby(['district','neighbourhood'])['monthly_earning_€'].mean().reset_index()
airbnbplot = airgroup.merge(latlong, how = 'inner', left_on=['district','neighbourhood'], right_on=['district','neighbourhood'])  
airbnbplot['monthly_earning_€'] = round(airbnbplot['monthly_earning_€'],2)

#Individual Sales Listings
salesgroup = madridcutsale.groupby(['district','neighbourhood'])['monthly_earning_€'].mean().reset_index()
saleplot = salesgroup.merge(latlong, how = 'inner', left_on=['district','neighbourhood'], right_on=['district','neighbourhood'])  
saleplot['monthly_earning_€'] = round(saleplot['monthly_earning_€'],2)

#Individual Rent Listings
rentgroup = madridcutrent.groupby(['district','neighbourhood'])['monthly_earning_€'].mean().reset_index()
rentplot = rentgroup.merge(latlong, how = 'inner', left_on=['district','neighbourhood'], right_on=['district','neighbourhood'])  
rentplot['monthly_earning_€'] = round(rentplot['monthly_earning_€'],2)
rentplot

#All Listings
group = madridcutrent.groupby(['district','neighbourhood'])['monthly_earning_€'].mean().reset_index()
plot = group.merge(latlong, how = 'inner', left_on=['district','neighbourhood'], right_on=['district','neighbourhood'])  
plot['monthly_earning_€'] = round(plot['monthly_earning_€'],2)
plot

#Indicator Groupby for Projected Earnings by district
indiprjes = madridcut.groupby(['district'])['monthly_earning_€'].sum().reset_index()
indiprjes['monthly_earning_€'] = round(indiprjes['monthly_earning_€']).astype(int)

#Indicator Groupby for Average price per m2 by district
indim2s = housing.groupby(['district'])['sq_m_price_€/m2'].mean().reset_index()
indim2s = round(indim2s['sq_m_price_€/m2'],2)

#Indicator for Average Airbnb Price
avrabs = airbnb.groupby(['district'])['monthly_earning_€'].mean().reset_index()
avrabs['monthly_earning_€'] = round(avrabs['monthly_earning_€'],2)

#Indicator for Average rent price 
avres = housing.groupby(['district'])['rent_price_€'].mean().reset_index()
avres['rent_price_€'] = round(avres['rent_price_€'],2)

#Bar Graph Neighbourhoods in district
barne = madridcut.groupby(['district','neighbourhood'])['monthly_earning_€'].mean().reset_index()
barne['monthly_earning_€'] = round(barne['monthly_earning_€'],2)


############################INITIATE DASH################################################


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY])

items = [{"label": (i), "value": i} for i in list(madridcut['district'].unique())]



################################PART 1 - LAYOUT OF DASH###################################



########### ROW 1 #############


#Create the first row, with the title, a link to linkedin, and today's date 
row1 = dbc.Row([
                        dbc.Col([
                            ##Photograph of linkedin icon with link inside, and caption.
                            dbc.CardLink([dbc.CardImg(src = 'https://salon-cprint.es/wp-content/uploads/2014/11/Linkedin1-300x300.png', style = {'width': '15%'}),
                                  html.H6('Created by Philip Kitchener')],
                                    href = 'https://www.linkedin.com/in/philipkitchener/')
                        ], style = {'text-align': 'center',
                                   'color' : '#e67339'}, width = 3, ),  
                            
                            
                        dbc.Col([
                            ##Title 
                            html.H2('AirBnB & Madrid', style = {'text-align' : 'center'}),
                            html.H5("A comparison between the value of Madrid's Housing Market & AirBnB", style = {'text-align' : 'center'})
                        ], style = {'text-align': 'center',
                                   'color' : '#e67339'}, width = 6),
                        
                        
                        dbc.Col([
                            ##Today's date
                            html.H6('Date : ' + str(datetime.date.today()), style = {'text-align' : 'center'})
                        ], className = 'mt-5', width = 3) # or to do the same thing: style = {'margin':25})
                    ])


########### ROW 2 #############


#Create the second row, with the "fact cards"
row2 = dbc.Row([
                    #Set the "fact card", which details the number of airbnbs
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                            html.H5('AirBnB in Madrid', className = 'card-title', style = {'text-align': 'center',
                                          'color' : 'Tomato'}),
                            html.P(madridcutairb['id'].count(),
                                  style = {'text-align': 'center',
                                          'color' : 'black',
                                          'fontSize': 25},
                                  className = 'card-text')
                            ])
                        ],inverse = True)
                      
                    ]),

                    #Set the "fact card", which details the number of rentals
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                            html.H5('Rentals in Madrid', className = 'card-title', style = {'text-align': 'center', 'color': 'blue'}),
                            html.P(housing['id'].count(),
                                  style = {'text-align': 'center',
                                          'color' : 'black',
                                          'fontSize': 25},
                                  className = 'card-text' )
                            ])
                        ], inverse = True)
                      
                    ])
    ])
    

########### ROW 3 #############


#Create the third row, with the dropdown menu (by district), donut chart, and bar chart (by neighbourhood)
row3 = dbc.Row([
                    dbc.Col([
                    dbc.Card([
                            dbc.CardBody([dbc.FormGroup([
                                dbc.Label("Select District", style = {'text-align' : 'right', 'color':'black'}),
                                #dropdown box initiation
                                        dcc.Dropdown(id = 'district', options = items, value = 'Arganzuela'),
                                        html.Br(),
                                        html.H6(("About The District"), style = {'text-align' : 'center'}),
                                        html.Br(),                                    
                                ])
                            ])                         
                        ], style = {'height': '80%'})
                    ], width = 3),
    
                    dbc.Col([                        
                        dbc.Card([
                            dbc.CardBody([
                                dcc.Graph( id = 'donut')
                                ])
                        ])
                            ],width = 4),
    
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                                dcc.Graph(id='bar')
                            ])
                        ])
                    ], width = 5)
                    
                ], className = 'my-3')


########### LAYOUT & ROW 4 #############
#n.b - when finishing the last row, it's important to determine the layout, so that dash knows it's the last row. 

#Determine layout
app.layout = dbc.Container([
                row1,
                row2,
                row3,
                #Create the fourth row, which includes four maps, depending on the button pushed.  
                dbc.Row([
                   
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                                #insert maps and buttons
                                dcc.Graph(id='madridmap'),
                                dbc.Button("AirBnB", id= "AirBnB", color="dark", className="m-1"),                            
                                dbc.Button("Rent",id="Rent", color="dark", className="m-1"),
                                dbc.Button("Sale", id="Sale", color="dark", className="m-1"),
                                dbc.Button("All Properties", id="All Properties", color="dark", className="m-1")                              
                                        ])
                                ], inverse = True)    
                            ], width = 12),
                        ])   
                            ], fluid = True)



################################PART 2 - Call & Define Graphs###################################

###Callback essentially serves to call the stated graphs from the function below. Input is the input data from the dropdown...
###...menu stated above, output being the graphs being put pack to the dashboard. 
@app.callback(
    Output(component_id='donut', component_property = 'figure'),
    Output(component_id='bar', component_property = 'figure'),
    Input(component_id='district', component_property = 'value')
)

def output_graphs(district):

    
    #####Donut Chart#####
    
    #For each section, determine the district by chosen district in dropdown menu
    madrent = madridcut[madridcut['type'] == 'rent']
    madren = madrent[madrent['district'] == district]
    
    madsal = madridcut[madridcut['type'] == 'sale']
    madsal = madsal[madsal['district'] == district]
    
    madair = madridcut[madridcut['type'] == 'airbnb']
    madair = madair[madair['district'] == district]
    
    #Initiate the donut. 

    donut = go.Figure()
    donut.add_trace(go.Pie(labels = ['AirBnB', 'Rent', 'Sale'],
                            values = [madrent['type'].count(),
                                      madrent['type'].count(),
                                      madsal['type'].count()
                                     ],
                            textinfo = 'label+value', textposition = 'outside')),
    donut.update_traces(marker_colors=['orange', 'red', '#66CC85'], hole = 0.6, rotation = 45),
    donut.update_layout(legend=dict(
            orientation="h",
             yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            ),template = "simple_white",
            title = {'text': 'Latest Information for {}:'.format(district),
                    'y': 0.95},
                    title_font_size = 12           
                        )
    
    
    #####Bar Chart##### 
    ##This will serve to show the different neighbourhoods within the district. 

    #Set district to chosen district from dropdown menu
    barno = barne[barne['district'] == district]
    bar = go.Figure()    
    bar.add_trace(go.Bar(
                x = barno["neighbourhood"], 
                y = barno["monthly_earning_€"],
                name = 'Performance of Neighbourhoods in {}:'.format(district),
                marker_color = 'Tomato',
        ))

    bar.update_layout(legend=dict(
            orientation="h",
             yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            ),
            )


    return donut, bar


###Set the inputs and outputs. Here i will use buttons to determine the information shown, this serves essentially
###as an input. 
@app.callback(
        Output(component_id='madridmap', component_property = 'figure'),
        Input(component_id='AirBnB', component_property = 'n_clicks'),
        Input(component_id='Rent', component_property = 'n_clicks'),
        Input(component_id='Sale', component_property = 'n_clicks'),
        Input(component_id='All Properties', component_property = 'n_clicks'))


##The maps below are of Airbnb lodgings, Rentals, Sales and All.
def buttons(btn1, btn2, btn3, btn4):        
    button_click = [p['prop_id'] for p in dash.callback_context.triggered][0]

    
    if button_click == '.' or 'AirBnb' in button_click:
        print('Airbnb Clicked'),
        
        madmap = go.Figure()

        madmap.add_trace(go.Scattermapbox(
                lat = airbnbplot['latitude'],
                lon = airbnbplot['longitude'],
                mode = 'markers',
                marker=go.scattermapbox.Marker(
                    size = airbnbplot['monthly_earning_€']/100,
                    color = airbnbplot['monthly_earning_€'],
                    colorscale = 'hsv',
                    showscale = False,
                    sizemode = 'area',
                    opacity = 0.3
                )
        ))
        
        madmap.update_layout(
            margin=dict(l=10, r=10, t=25, b=2),
            template = "plotly_dark",
           # paper_bgcolor = '#282828',
            hovermode='closest',
            title = 'AirBnB Earnings in Madrid',
            mapbox=dict(
                accesstoken='pk.eyJ1Ijoia2l0Y2hwaGlsIiwiYSI6ImNrcDVueHE1MTAwdm8ydm1scXpwbzBqdDYifQ.mgRPiB0FLJErcM5L_cqhjg',
                bearing= 0,
                center =dict(lat=40.42, lon=-3.68),
                zoom=11,
            )
        )
 
        #return madmap


    elif 'Rent' in button_click:
        print('Rent Clicked')
               
        madmap = go.Figure()

        madmap.add_trace(go.Scattermapbox(
                lat = rentplot['latitude'],
                lon = rentplot['longitude'],
                mode = 'markers',
                marker=go.scattermapbox.Marker(
                    size = rentplot['monthly_earning_€']/100,
                    color = rentplot['monthly_earning_€'],
                    colorscale = 'hsv',
                    showscale = False,
                    sizemode = 'area',
                    opacity = 0.3
                )
        ))

        
        madmap.update_layout(
            margin=dict(l=10, r=10, t=25, b=2),
            template = "plotly_dark",
            #paper_bgcolor = '#282828',
            hovermode='closest',
            title = 'Rent Earnings in Madrid',
            mapbox=dict(
                accesstoken='pk.eyJ1Ijoia2l0Y2hwaGlsIiwiYSI6ImNrcDVueHE1MTAwdm8ydm1scXpwbzBqdDYifQ.mgRPiB0FLJErcM5L_cqhjg',
                bearing= 0,
                center =dict(lat=40.42, lon=-3.68),
                zoom=11,
            )
        )
        
        #return madmap
        
    elif 'Sale' in button_click:
        print('Sale Clicked')
        
        madmap = go.Figure()

        madmap.add_trace(go.Scattermapbox(
                lat = saleplot['latitude'],
                lon = saleplot['longitude'],
                mode = 'markers',
                marker=go.scattermapbox.Marker(
                    size = saleplot['monthly_earning_€']/100,
                    color = saleplot['monthly_earning_€'],
                    colorscale = 'hsv',
                    showscale = False,
                    sizemode = 'area',
                    opacity = 0.3
                )

        ))

        madmap.update_layout(
            margin=dict(l=10, r=10, t=25, b=2),
            template = "plotly_dark",
            #paper_bgcolor = '#282828',
            hovermode='closest',
            title = 'Houses on Sale in Madrid',
            mapbox=dict(
                accesstoken='pk.eyJ1Ijoia2l0Y2hwaGlsIiwiYSI6ImNrcDVueHE1MTAwdm8ydm1scXpwbzBqdDYifQ.mgRPiB0FLJErcM5L_cqhjg',
                bearing= 0,
                center =dict(lat=40.42, lon=-3.68),
                zoom=11,
            )
        )
 
        #return madmap
    
    
    elif 'All Properties' in button_click:
        print('Sale Clicked')
    
        madmap = go.Figure()

        madmap.add_trace(go.Scattermapbox(
                lat = plot['latitude'],
                lon = plot['longitude'],
                mode = 'markers',
                marker=go.scattermapbox.Marker(
                    size = plot['monthly_earning_€']/100,
                    color = plot['monthly_earning_€'],
                    colorscale = 'hsv',
                    showscale = False,
                    sizemode = 'area',
                    opacity = 0.3
                )

        ))

        madmap.update_layout(
            margin=dict(l=10, r=10, t=25, b=2),
            template = "plotly_dark",
            #paper_bgcolor = '#282828',
            hovermode='closest',
            title = 'All Property Types in Madrid',
            mapbox=dict(
                accesstoken='pk.eyJ1Ijoia2l0Y2hwaGlsIiwiYSI6ImNrcDVueHE1MTAwdm8ydm1scXpwbzBqdDYifQ.mgRPiB0FLJErcM5L_cqhjg',
                bearing= 0,
                center =dict(lat=40.42, lon=-3.68),
                zoom=11,
            )
        )
 
    return madmap


##Run the Dash    
if __name__ == '__main__':
    app.run_server()